<br>

# Maps

In [ ]:
#!pip3 install open-geodata --upgrade
#!pip3 install sp-piracicaba

In [ ]:
import folium
import geopandas as gpd
import open_geodata as geo
from folium import plugins
from paths import *

In [ ]:
# import json
# import os
# from datetime import date
# import os
# import webbrowser
# import branca as bc
# import fiona
# import pandas as pd
# import seaborn as sns
# import sp_piracicaba.lyr

<br>

## Maps

In [ ]:
#list_shp = geo.get_dataset_from_package('sp_piracicaba')
#list_shp

In [ ]:
# List geodataframes
# geo.get_dataset_names()
gdf = geo.load_dataset(db='sp_piracicaba', name='geo.divisa_municipal')

In [ ]:

def macrozona():
    # Input
    gdf = geo.load_dataset_from_package('sp_piracicaba', 'geo.macrozonas')
    gdf = gdf.to_crs(epsg=4326)

    # Set dictionary
    colors = {
        'MADE': '#0173b2',
        'MANU': '#de8f05',
        'MAPH': '#029e73',
        'MCU': '#d55e00',
        'MRU': '#cc78bc',
        'MUC': '#ca9161'
    }

    # Popup
    gdf['PopUp'] = gdf.apply(popup_macrozona, axis=1)

    # Layer
    return folium.features.GeoJson(
        gdf,
        name='Macrozoneamento',
        style_function=lambda x: {
            'fillColor': colors[x['properties']['Macrozona']],
            'color': colors[x['properties']['Macrozona']],
            'weight': 0,
            'fillOpacity': 0.3
        },
        popup=folium.GeoJsonPopup(
            fields=['PopUp'],
            aliases=['Macrozona'],
            labels=False,
            sticky=True,
            localize=True,
            style=f"""
            background-color: #F0EFEF;
            """,
            parse_html=True,
            max_width=400,
        ),
        tooltip=folium.features.GeoJsonTooltip(
            fields=['Macrozona', ],
            aliases=['Macrozona', ],
            labels=True,
            sticky=True,
            opacity=0.9,
            direction='auto',
        ),
        highlight_function=lambda x: {
            'weight': 3
        },
        embed=False,
        zoom_on_click=False,
        control=True,
        show=False,
    )


# PopUp
def popup_macrozona(row):
    descriptions = {
        'MADE': 'Macrozona de Desenvolvimento Rural',
        'MANU': 'Macrozona de Núcleos Urbanos Isolados',
        'MAPH': 'Macrozona de Proteção Hídrica e Ambiental',
        'MCU': 'Macrozona de Contenção Urbana',
        'MRU': 'Macrozona de Restrição Urbana',
        'MUC': 'Macrozona de Urbanização Consolidada',
    }
    macrozona = row['Macrozona']
    description = descriptions[macrozona]
    return f"""
    <div>
    <h4>Plano Diretor de Piracicaba</h4>
    <h5>Lei Complementar nº 405/2019</h5>
    <b>{description}</b>
    </div>    
    """
    


def perimetro_urbano():
    # Input
    gdf = geo.load_dataset_from_package('sp_piracicaba', 'geo.divisa_perimetro')
    gdf = gdf.to_crs(epsg=4326)

    # Layer
    return folium.features.GeoJson(
        gdf,
        name='Perimetro Urbano',
        style_function=lambda x: {
            'fillColor': '#E1E1E1',
            'color': '#E1E1E1',
            'weight': 2,
            'fillOpacity': 0.5,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['Nome', ],
            aliases=['Nome', ],
            labels=True,
            sticky=True,
            opacity=0.9,
            direction='auto',
        ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=False,
    )


def divisa_municipal():
    # Input
    gdf = geo.load_dataset_from_package('sp_piracicaba', 'geo.divisa_municipal')
    gdf = gdf.to_crs(epsg=4326)

    # Layer
    return folium.features.GeoJson(
        gdf,
        name='Divisão Municipal',
        style_function=lambda x: {
            'fillColor': '#FAE878',
            'color': '#FAE878',
            'weight': 2,
            'fillOpacity': 0.0,
        },
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )


def divisa_urbano_rural():
    # Input
    gdf = geo.load_dataset_from_package('sp_piracicaba', 'geo.divisa_urbanorural')
    gdf = gdf.to_crs(epsg=4326)

    # Set dictionary
    colors = {
        'Área Rural': '#F1A51F',
        'Área Urbana': '##E1E1E1',
    }

    # Layer
    return folium.features.GeoJson(
        gdf,
        name='Divisão Urbano Rural',
        style_function=lambda x: {
            'fillColor': colors[x['properties']['Area']],
            'color': colors[x['properties']['Area']],
            'weight': 1,
            'fillOpacity': 0.3
        },
        embed=False,
        zoom_on_click=False,
        control=True,
        show=False,
    )


def divisa_abairramento():
    # Input
    gdf = geo.load_dataset_from_package('sp_piracicaba', 'geo.divisa_abairramento')
    gdf = gdf.to_crs(epsg=4326)

    # Layer
    return folium.features.GeoJson(
        gdf,
        name='Divisão Abairramento',
        style_function=lambda x: {
            'fillColor': '#CFCFCF',
            'color': '#CFCFCF',
            'weight': 1,
            'fillOpacity': 0.3
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NomeBairro'],
            aliases=['Bairro'],
            labels=True,
            sticky=True,
            opacity=0.9,
            direction='auto',
        ),
        highlight_function=lambda x: {
            'weight': 3
        },
        embed=False,
        zoom_on_click=False,
        control=True,
        show=False,
    )


<br>

# Mapa Plano Diretor

In [ ]:
def get_map(input_shp):
    # Input
    gdf = gpd.read_file(input_shp)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 10
    max_zoom = 18

    # Create Map
    m = folium.Map(
        #location=list_centroid,
        #zoom_start=10,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        #zoom_delta=0.1,
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )
    
    # Add Base Layers
    m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(lyr.base.google_streets(min_zoom, max_zoom))
    m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))
    
    # Plano Diretor
    m.add_child(sp_piracicaba.lyr.divisa_municipal())
    m.add_child(sp_piracicaba.lyr.macrozona())
    m.add_child(sp_piracicaba.lyr.divisa_abairramento())
    m.add_child(sp_piracicaba.lyr.divisa_urbano_rural())
    m.add_child(sp_piracicaba.lyr.perimetro_urbano())
    
    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False
    ).add_to(m)
    return m

In [ ]:
m = get_map(output_path_shp / 'Divisa_Municipal.shp')

# Results
m.save(output_path_map / 'map_pd_piracicaba.html')
m